In [1]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def inite(elec, orb):
    list1 = []
    # Single excitations
    list1.append([e for e in range(elec)])
    for x in range(elec):
        config = []
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    config.append(count + (1 if x % 2 else 0))
                else:
                    config.append(e)
            list1.append(config)
            config = []
            count += 2
    # Double excitations
    for x in range(elec):
        for y in range(x + 1, elec):
            for count1 in range(orb - elec, orb, 2):
                for count2 in range(orb - elec, orb, 2):
                    cont = 1 if (count1 != count2 or (x % 2) != (y % 2)) else 0
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        config = []
                        for e in range(elec):
                            if x == e:
                                config.append(count1 + (1 if x % 2 else 0))
                            elif y == e:
                                config.append(count2 + (1 if y % 2 else 0))
                            else:
                                config.append(e)
                        list1.append(config)
    return list1

def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")
    hf_state = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
    dev = qml.device("default.qubit", wires=qubits, shots=shots)

    if shots is None:
        @qml.qnode(dev, interface="autograd", diff_method="adjoint")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            for i, ex in enumerate(pool):
                if len(ex) == 4:
                    qml.FermionicDoubleExcitation(params[i], wires1=[ex[0], ex[1]], wires2=[ex[2], ex[3]])
                elif len(ex) == 2:
                    qml.FermionicSingleExcitation(params[i], wires=[ex[0], ex[1]])
            return qml.expval(H)
    else:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
geometry = np.array([[0.0, 0.0,  0.0],                       
                    [0.0, 0.0, 1.5*r],
                    [0.0, 1.5*r, 0.0],
                    [0.0, 1.5*r, 1.5*r]
                    
                    ], requires_grad = False)
electrons = 4
charge = 0
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=500)

def qsceom(symbols, geometry, electrons, charge, params, shots=0, p=0.1):
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires = list(range(qubits))
    hf_state = qml.qchem.hf_state(electrons, qubits)
    null_state = np.zeros(qubits, int)

    list1 = inite(electrons, qubits)
    values = []

    # Device
    dev = qml.device("lightning.qubit", wires=qubits, shots=shots if shots > 0 else None)

    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state, pool):
        qml.BasisState(hf_state, wires=wires)
        for w in occ:
            qml.X(wires=w)
        for i, ex in enumerate(pool):
            if len(ex) == 4:
                qml.FermionicDoubleExcitation(params[i], wires1=[ex[0], ex[1]], wires2=[ex[2], ex[3]])
            elif len(ex) == 2:
                qml.FermionicSingleExcitation(params[i], wires=[ex[0], ex[1]])
        #qml.BitFlip(p=0.0, wires=0)
        return qml.expval(H)

    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state, pool):
        for w in occ1:
            qml.X(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for i, ex in enumerate(pool):
            if len(ex) == 4:
                qml.FermionicDoubleExcitation(params[i], wires1=[ex[0], ex[1]], wires2=[ex[2], ex[3]])
            elif len(ex) == 2:
                qml.FermionicSingleExcitation(params[i], wires=[ex[0], ex[1]])
        #qml.BitFlip(p=0.0, wires=0)
        return qml.expval(H)

    M = np.zeros((len(list1), len(list1)))
    for i in range(len(list1)):
        M[i, i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state, pool)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i != j:
                Mtmp = circuit_od(params, list1[i], list1[j], wires, s_wires, d_wires, null_state, pool)
                M[i, j] = Mtmp - M[i, i] / 2.0 - M[j, j] / 2.0
    eig = np.linalg.eig(M)
    values.append(np.sort(eig[0]))
    return values

eig = qsceom(symbols, geometry, electrons, charge, params, shots=0, p=0.0)
print("excited state energies:\n", eig)


Ground state energy: 1.91004837 Ha
excited state energies:
 [tensor([-1.92330563, -1.90091179, -1.7725946 , -1.73233984, -1.72662792,
        -1.70223976, -1.61641809, -1.43178969, -1.43103173, -1.42395431,
        -1.42351939, -1.32667878, -1.28462352, -1.26955888, -1.25751614,
        -1.2266879 , -1.21791879, -1.21485308, -1.12149876, -1.11812447,
        -1.02976886, -1.00545908, -0.79998911, -0.72164557, -0.66490044,
        -0.66091371, -0.50341091], requires_grad=True)]


In [2]:
Ground state energy: -1.95195624 Ha
excited state energies:
 [tensor([-1.93244626, -1.91356754, -1.78917618, -1.7831505 , -1.70918491,
        -1.63530371, -1.50067561, -1.43079623, -1.42373963, -1.41288145,
        -1.40482551, -1.33794933, -1.31894904, -1.27151697, -1.22765151,
        -1.21416396, -1.18627814, -1.12735451, -1.12134456, -1.07878575,
        -1.01963289, -1.00620087, -0.82188556, -0.74922978, -0.67775314,
        -0.63698701, -0.57377481], requires_grad=True)]

SyntaxError: invalid syntax (1312680221.py, line 1)

In [ ]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def inite(elec, orb):
    list1 = []
    # Single excitations
    list1.append([e for e in range(elec)])
    for x in range(elec):
        config = []
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    config.append(count + (1 if x % 2 else 0))
                else:
                    config.append(e)
            list1.append(config)
            config = []
            count += 2
    # Double excitations
    for x in range(elec):
        for y in range(x + 1, elec):
            for count1 in range(orb - elec, orb, 2):
                for count2 in range(orb - elec, orb, 2):
                    cont = 1 if (count1 != count2 or (x % 2) != (y % 2)) else 0
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        config = []
                        for e in range(elec):
                            if x == e:
                                config.append(count1 + (1 if x % 2 else 0))
                            elif y == e:
                                config.append(count2 + (1 if y % 2 else 0))
                            else:
                                config.append(e)
                        list1.append(config)
    return list1

def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")
    hf_state = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
    dev = qml.device("default.qubit", wires=qubits, shots=shots)

    if shots is None:
        @qml.qnode(dev, interface="autograd", diff_method="adjoint")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
    else:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
geometry = np.array([[0.0, 0.0,  0.0],                       
                    [0.0, 0.0, 1.5*r],
                    [0.0, 1.5*r, 0.0],
                    [0.0, 1.5*r, 1.5*r]
                    
                    ], requires_grad = False)
electrons = 4
charge = 0
params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)

def qsceom(symbols, geometry, electrons, charge, params, shots=0, p=0.1):
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires = list(range(qubits))
    hf_state = qml.qchem.hf_state(electrons, qubits)
    null_state = np.zeros(qubits, int)

    list1 = inite(electrons, qubits)
    values = []

    # Device
    dev = qml.device("default.mixed", wires=qubits, shots=shots if shots > 0 else None)

    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state, pool):
        qml.BasisState(hf_state, wires=wires)
        for w in occ:
            qml.X(wires=w)
        for i, ex in enumerate(pool):
            if len(ex) == 4:
                qml.FermionicDoubleExcitation(params[i], wires1=[ex[0], ex[1]], wires2=[ex[2], ex[3]])
            elif len(ex) == 2:
                qml.FermionicSingleExcitation(params[i], wires=[ex[0], ex[1]])
        qml.BitFlip(p=0.1, wires=0)
        return qml.expval(H)

    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state, pool):
        for w in occ1:
            qml.X(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for i, ex in enumerate(pool):
            if len(ex) == 4:
                qml.FermionicDoubleExcitation(params[i], wires1=[ex[0], ex[1]], wires2=[ex[2], ex[3]])
            elif len(ex) == 2:
                qml.FermionicSingleExcitation(params[i], wires=[ex[0], ex[1]])
        qml.BitFlip(p=0.1, wires=0)
        return qml.expval(H)

    M = np.zeros((len(list1), len(list1)))
    for i in range(len(list1)):
        M[i, i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state, pool)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i != j:
                Mtmp = circuit_od(params, list1[i], list1[j], wires, s_wires, d_wires, null_state, pool)
                M[i, j] = Mtmp - M[i, i] / 2.0 - M[j, j] / 2.0
    eig = np.linalg.eig(M)
    values.append(np.sort(eig[0]))
    return values

eig = qsceom(symbols, geometry, electrons, charge, params, shots=0, p=0.0)
print("excited state energies:\n", eig)


Ground state energy: -1.95195624 Ha
